In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [64]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [65]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [66]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
# X.columns

In [67]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X[col_names] = features

In [68]:

features = X_test[col_names]
scaler_test = StandardScaler().fit(features.values)
features = scaler_test.transform(features.values)
X_test[col_names] = features

### model creation


In [69]:
def model_accuracy(selected_features,X,X_test,y):
    X = X[selected_features]
    X_test = X_test[selected_features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    sm = SMOTE(sampling_strategy='minority', random_state = 42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    kfold = StratifiedKFold(n_splits=3, shuffle = False)
    def evaluate(model, X_train, y_train, X_test, y_test):
        print("TRAIN")
        train_predictions = model.predict_proba(X_train)
        train_pred = model.predict(X_train)

        train_accuracy = accuracy_score(y_train, train_pred)
        train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])

        print("Classification report")
        print(classification_report(y_train, train_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(train_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
        print("*" * 100)

        print("TEST")
        test_predictions = model.predict_proba(X_test)
        test_pred = model.predict(X_test)

        test_accuracy = accuracy_score(y_test, test_pred)
        test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])

        print("Classification report")
        print(classification_report(y_test, test_pred, digits = 4))

        print("FBeta Score")
        print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))

        print('Model Performance')
        print('Accuracy = {:0.4f}%.'.format(test_accuracy))
        print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
        print("*" * 100)
    def create_model(learn_rate=0.01, momentum=0):
        # create model
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    print("BASED MODEL")
    ann_accuracy = evaluate(ann, X_train, y_train, X_test, y_test)

    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = 2, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    rand_best_params = ann_random.best_params_
    print(ann_random.best_params_)
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)


    print("GRID SEARCH ANN EXPERIMENT")
    #     from random search
    #     'momentum': 0.0, 'learn_rate': 0.3, 'epochs': 100, 'batch_size': 10
    # learn_rate = [0.3, 0.1, 0.2]
    # momentum = [0.0, 0.2, 0.4]
    # batch_size = [5, 10, 20]
    # epochs = [50, 100, 150]
    learn_rate = [rand_best_params['learn_rate'],rand_best_params['learn_rate']*1.5,rand_best_params['learn_rate']/1.5]
    momentum = [rand_best_params['momentum'],rand_best_params['momentum']*1.5,rand_best_params['momentum']/1.5]
    batch_size = [rand_best_params['batch_size'],rand_best_params['batch_size']*1.5,rand_best_params['batch_size']/1.5]
    epochs = [rand_best_params['epochs'],rand_best_params['epochs']*1.5,rand_best_params['epochs']/1.5]
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    # ann_grid = rand_best_params
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=2, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    ann_best_grid = ann_grid.best_estimator_
    grid_best_parms = ann_grid.best_params_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)


# Other number of Features

In [78]:
selected_features_10 = ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile']
selected_features_15 = ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry']
selected_features_20 =['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy']
selected_features_25= ['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction']
selected_features_30=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction']
selected_features_35=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction']
selected_features_40=['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy', 'on_ip_that_has_a_bot_mean', 'device', 'home goods', 'time', 'std_url_per_auction', 'country', 'sporting goods', 'num_bids', 'perc_inst_resp', 'max_country_per_auction', 'mean_ip_per_auction', 'std_devices_per_auction', 'max_url_per_auction', 'std_ip_per_auction', 'mean_devices_per_auction', 'mean_url_per_auction', 'total_no_of_participated_auctions', 'std_country_per_auction', 'ip_entropy', 'mean_country_per_auction']



In [79]:
model_accuracy(selected_features_10,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8236    0.6969    0.7550      1333
         1.0     0.7373    0.8507    0.7900      1333

    accuracy                         0.7738      2666
   macro avg     0.7805    0.7738    0.7725      2666
weighted avg     0.7805    0.7738    0.7725      2666

FBeta Score
0.8253275109170306
Model Performance
Accuracy = 0.7738%.
AUC ROC = 0.8338%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9927    0.7036    0.8235       577
         1.0     0.1231    0.8889    0.2162        27

    accuracy                         0.7119       604
   macro avg     0.5579    0.7963    0.5199       604
weighted avg     0.9538    0.7119    0.7964       604

FBeta Score
0.39603960396039606
Model Performance
Accuracy = 0.7119%.
AUC ROC = 0.8352%.
*

2021-11-06 01:25:36.602134: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 01:25:36.602137: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 01:25:36.602133: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

{'momentum': 0.9, 'learn_rate': 0.3, 'epochs': 100, 'batch_size': 10}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9940    0.8770    0.9318      1333
         1.0     0.8899    0.9947    0.9394      1333

    accuracy                         0.9359      2666
   macro avg     0.9420    0.9359    0.9356      2666
weighted avg     0.9420    0.9359    0.9356      2666

FBeta Score
0.9718557607739666
Model Performance
Accuracy = 0.9359%.
AUC ROC = 0.9861%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9760    0.8440    0.9052       577
         1.0     0.1429    0.5556    0.2273        27

    accuracy                         0.8311       604
   macro avg     0.5594    0.6998    0.5662       604
weighted avg     0.9387    0.8311    0.8749       604

FBeta Score
0.352112676056338
Mo

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9888    0.9280    0.9574      1333
         1.0     0.9322    0.9895    0.9600      1333

    accuracy                         0.9587      2666
   macro avg     0.9605    0.9587    0.9587      2666
weighted avg     0.9605    0.9587    0.9587      2666

FBeta Score
0.9774714688009487
Model Performance
Accuracy = 0.9587%.
AUC ROC = 0.9932%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9809    0.8891    0.9327       577
         1.0     0.2099    0.6296    0.3148        27

    accuracy                         0.8775       604
   macro avg     0.5954    0.7594    0.6238       604
weighted avg     0.9464    0.8775    0.9051       604

FBeta Score
0.44973544973544977
Model Performance
Accuracy = 0.8775%.
AUC ROC = 0.7357%.
*************

[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.2; total time=  40.3s
[CV] END batch_size=20, epochs=10, learn_rate=0.3, momentum=0.6; total time=   7.3s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=  42.9s
[CV] END batch_size=60, epochs=50, learn_rate=0.01, momentum=0.9; total time=   4.7s
[CV] END batch_size=60, epochs=100, learn_rate=0.2, momentum=0.2; total time=  22.8s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.0; total time=  30.3s
[CV] END batch_size=10, epochs=50, learn_rate=0.001, momentum=0.0; total time=  15.2s
[CV] END batch_size=80, epochs=100, learn_rate=0.3, momentum=0.4; total time=   4.9s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.8; total time=  29.9s
[CV] END batch_size=60, epochs=100, learn_rate=0.001, momentum=0.9; total time=   8.1s
[CV] END batch_size=10, epochs=100, learn_rate=0.01, momentum=0.4; total time=  30.0s
[CV] END batch_size=10, epochs=100, learn_rate=0.3, momentum=0

In [70]:
model_accuracy(selected_features_15,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.6495    0.8980    0.7538      1333
         1.0     0.8348    0.5154    0.6373      1333

    accuracy                         0.7067      2666
   macro avg     0.7421    0.7067    0.6955      2666
weighted avg     0.7421    0.7067    0.6955      2666

FBeta Score
0.5580828594638506
Model Performance
Accuracy = 0.7067%.
AUC ROC = 0.6242%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9705    0.9116    0.9401       577
         1.0     0.1774    0.4074    0.2472        27

    accuracy                         0.8891       604
   macro avg     0.5739    0.6595    0.5937       604
weighted avg     0.9350    0.8891    0.9091       604

FBeta Score
0.32352941176470584
Model Performance
Accuracy = 0.8891%.
AUC ROC = 0.5259%.
*

2021-11-05 23:55:09.309906: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 23:55:09.309907: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 23:55:09.309909: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

{'momentum': 0.4, 'learn_rate': 0.01, 'epochs': 100, 'batch_size': 10}
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9731    0.8957    0.9328      1333
         1.0     0.9034    0.9752    0.9380      1333

    accuracy                         0.9355      2666
   macro avg     0.9383    0.9355    0.9354      2666
weighted avg     0.9383    0.9355    0.9354      2666

FBeta Score
0.9599763698124353
Model Performance
Accuracy = 0.9355%.
AUC ROC = 0.9811%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9724    0.8544    0.9096       577
         1.0     0.1340    0.4815    0.2097        27

    accuracy                         0.8377       604
   macro avg     0.5532    0.6680    0.5596       604
weighted avg     0.9349    0.8377    0.8783       604

FBeta Score
0.31707317073170727

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9706    0.8912    0.9292      1333
         1.0     0.8994    0.9730    0.9348      1333

    accuracy                         0.9321      2666
   macro avg     0.9350    0.9321    0.9320      2666
weighted avg     0.9350    0.9321    0.9320      2666

FBeta Score
0.9573368762917034
Model Performance
Accuracy = 0.9321%.
AUC ROC = 0.9792%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9764    0.8596    0.9143       577
         1.0     0.1562    0.5556    0.2439        27

    accuracy                         0.8460       604
   macro avg     0.5663    0.7076    0.5791       604
weighted avg     0.9397    0.8460    0.8843       604

FBeta Score
0.36764705882352944
Model Performance
Accuracy = 0.8460%.
AUC ROC = 0.7062%.
*************

In [71]:
model_accuracy(selected_features_20,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8647    0.7142    0.7823      1333
         1.0     0.7565    0.8882    0.8171      1333

    accuracy                         0.8012      2666
   macro avg     0.8106    0.8012    0.7997      2666
weighted avg     0.8106    0.8012    0.7997      2666

FBeta Score
0.8583442076265042
Model Performance
Accuracy = 0.8012%.
AUC ROC = 0.8531%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9927    0.7054    0.8247       577
         1.0     0.1237    0.8889    0.2172        27

    accuracy                         0.7136       604
   macro avg     0.5582    0.7971    0.5210       604
weighted avg     0.9538    0.7136    0.7976       604

FBeta Score
0.3973509933774835
Model Performance
Accuracy = 0.7136%.
AUC ROC = 0.8070%.
**

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8052    0.9115    0.8550      1333
         1.0     0.8980    0.7794    0.8345      1333

    accuracy                         0.8455      2666
   macro avg     0.8516    0.8455    0.8448      2666
weighted avg     0.8516    0.8455    0.8448      2666

FBeta Score
0.8005856064108492
Model Performance
Accuracy = 0.8455%.
AUC ROC = 0.9009%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9624    0.8873    0.9234       577
         1.0     0.0972    0.2593    0.1414        27

    accuracy                         0.8593       604
   macro avg     0.5298    0.5733    0.5324       604
weighted avg     0.9237    0.8593    0.8884       604

FBeta Score
0.19444444444444445
Model Performance
Accuracy = 0.8593%.
AUC ROC = 0.5461%.
*************

In [72]:
model_accuracy(selected_features_25,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.6147    0.9227    0.7379      1333
         1.0     0.8451    0.4216    0.5626      1333

    accuracy                         0.6722      2666
   macro avg     0.7299    0.6722    0.6502      2666
weighted avg     0.7299    0.6722    0.6502      2666

FBeta Score
0.46856761714190437
Model Performance
Accuracy = 0.6722%.
AUC ROC = 0.5785%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9671    0.9168    0.9413       577
         1.0     0.1579    0.3333    0.2143        27

    accuracy                         0.8907       604
   macro avg     0.5625    0.6251    0.5778       604
weighted avg     0.9309    0.8907    0.9088       604

FBeta Score
0.27272727272727276
Model Performance
Accuracy = 0.8907%.
AUC ROC = 0.4849%.


/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9788    0.8650    0.9184      1333
         1.0     0.8790    0.9812    0.9273      1333

    accuracy                         0.9231      2666
   macro avg     0.9289    0.9231    0.9228      2666
weighted avg     0.9289    0.9231    0.9228      2666

FBeta Score
0.9589442815249268
Model Performance
Accuracy = 0.9231%.
AUC ROC = 0.9603%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9837    0.8371    0.9045       577
         1.0     0.1681    0.7037    0.2714        27

    accuracy                         0.8311       604
   macro avg     0.5759    0.7704    0.5880       604
weighted avg     0.9472    0.8311    0.8762       604

FBeta Score
0.4298642533936652
Model Performance
Accuracy = 0.8311%.
AUC ROC = 0.7873%.
**************

In [73]:
model_accuracy(selected_features_30,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.6248    0.7119    0.6655      1333
         1.0     0.6652    0.5724    0.6153      1333

    accuracy                         0.6422      2666
   macro avg     0.6450    0.6422    0.6404      2666
weighted avg     0.6450    0.6422    0.6404      2666

FBeta Score
0.5888254360240779
Model Performance
Accuracy = 0.6422%.
AUC ROC = 0.5939%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9752    0.6828    0.8033       577
         1.0     0.0850    0.6296    0.1498        27

    accuracy                         0.6805       604
   macro avg     0.5301    0.6562    0.4765       604
weighted avg     0.9355    0.6805    0.7741       604

FBeta Score
0.275974025974026
Model Performance
Accuracy = 0.6805%.
AUC ROC = 0.6368%.
***

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9766    0.8777    0.9245      1333
         1.0     0.8890    0.9790    0.9318      1333

    accuracy                         0.9284      2666
   macro avg     0.9328    0.9284    0.9282      2666
weighted avg     0.9328    0.9284    0.9282      2666

FBeta Score
0.9595588235294118
Model Performance
Accuracy = 0.9284%.
AUC ROC = 0.9607%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9863    0.8752    0.9275       577
         1.0     0.2174    0.7407    0.3361        27

    accuracy                         0.8692       604
   macro avg     0.6019    0.8080    0.6318       604
weighted avg     0.9520    0.8692    0.9010       604

FBeta Score
0.49999999999999994
Model Performance
Accuracy = 0.8692%.
AUC ROC = 0.8434%.
*************

In [74]:
model_accuracy(selected_features_35,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.6016    0.7997    0.6866      1333
         1.0     0.7013    0.4704    0.5631      1333

    accuracy                         0.6350      2666
   macro avg     0.6515    0.6350    0.6249      2666
weighted avg     0.6515    0.6350    0.6249      2666

FBeta Score
0.5035335689045937
Model Performance
Accuracy = 0.6350%.
AUC ROC = 0.5237%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9736    0.7678    0.8585       577
         1.0     0.1007    0.5556    0.1705        27

    accuracy                         0.7583       604
   macro avg     0.5371    0.6617    0.5145       604
weighted avg     0.9346    0.7583    0.8278       604

FBeta Score
0.2918287937743191
Model Performance
Accuracy = 0.7583%.
AUC ROC = 0.6094%.
**

/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9689    0.8425    0.9013      1333
         1.0     0.8607    0.9730    0.9134      1333

    accuracy                         0.9077      2666
   macro avg     0.9148    0.9077    0.9073      2666
weighted avg     0.9148    0.9077    0.9073      2666

FBeta Score
0.948238046498026
Model Performance
Accuracy = 0.9077%.
AUC ROC = 0.9501%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9854    0.8180    0.8939       577
         1.0     0.1600    0.7407    0.2632        27

    accuracy                         0.8146       604
   macro avg     0.5727    0.7794    0.5785       604
weighted avg     0.9485    0.8146    0.8657       604

FBeta Score
0.4291845493562232
Model Performance
Accuracy = 0.8146%.
AUC ROC = 0.8158%.
***************

In [75]:
model_accuracy(selected_features_40,X,X_test,y)

BASED MODEL
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.5543    0.9302    0.6947      1333
         1.0     0.7832    0.2521    0.3814      1333

    accuracy                         0.5911      2666
   macro avg     0.6688    0.5911    0.5380      2666
weighted avg     0.6688    0.5911    0.5380      2666

FBeta Score
0.29161603888213855
Model Performance
Accuracy = 0.5911%.
AUC ROC = 0.3727%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9640    0.9272    0.9452       577
         1.0     0.1429    0.2593    0.1842        27

    accuracy                         0.8974       604
   macro avg     0.5534    0.5932    0.5647       604
weighted avg     0.9273    0.8974    0.9112       604

FBeta Score
0.22292993630573243
Model Performance
Accuracy = 0.8974%.
AUC ROC = 0.3627%.


/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 219, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/brennanszeto/Environments/dsenv/lib/python3.9/site-packages/

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.7934    0.8642    0.8273      1333
         1.0     0.8509    0.7749    0.8112      1333

    accuracy                         0.8196      2666
   macro avg     0.8221    0.8196    0.8192      2666
weighted avg     0.8221    0.8196    0.8192      2666

FBeta Score
0.7890314695997556
Model Performance
Accuracy = 0.8196%.
AUC ROC = 0.8777%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9744    0.8579    0.9124       577
         1.0     0.1458    0.5185    0.2276        27

    accuracy                         0.8427       604
   macro avg     0.5601    0.6882    0.5700       604
weighted avg     0.9374    0.8427    0.8818       604

FBeta Score
0.3431372549019608
Model Performance
Accuracy = 0.8427%.
AUC ROC = 0.6715%.
**************

[CV] END batch_size=10, epochs=100, learn_rate=0.3, momentum=0.4; total time=  26.6s
[CV] END batch_size=20, epochs=10, learn_rate=0.3, momentum=0.6; total time=   3.1s
[CV] END batch_size=60, epochs=10, learn_rate=0.001, momentum=0.4; total time=   1.8s
[CV] END batch_size=20, epochs=10, learn_rate=0.01, momentum=0.9; total time=   2.9s
[CV] END batch_size=80, epochs=10, learn_rate=0.1, momentum=0.2; total time=   2.2s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.8; total time=   2.2s
[CV] END batch_size=20, epochs=100, learn_rate=0.1, momentum=0.9; total time=  14.4s
[CV] END batch_size=60, epochs=50, learn_rate=0.01, momentum=0.9; total time=   3.4s
[CV] END batch_size=60, epochs=100, learn_rate=0.2, momentum=0.2; total time=   7.6s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.8; total time=   2.4s
[CV] END batch_size=40, epochs=100, learn_rate=0.3, momentum=0.8; total time=   8.0s
[CV] END batch_size=10, epochs=100, learn_rate=0.2, momentum=0.2; to

[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   5.0s
[CV] END batch_size=100, epochs=10, learn_rate=0.2, momentum=0.2; total time=   1.9s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.6; total time=   2.7s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.8; total time=   4.7s
[CV] END batch_size=60, epochs=100, learn_rate=0.1, momentum=0.0; total time=   7.5s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   2.1s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.0; total time=   4.2s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=  25.0s
[CV] END batch_size=80, epochs=10, learn_rate=0.01, momentum=0.9; total time=   1.6s
[CV] END batch_size=100, epochs=10, learn_rate=0.1, momentum=0.4; total time=   2.5s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.4; total time=   6.1s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.0;

[CV] END batch_size=10, epochs=100, learn_rate=0.3, momentum=0.4; total time=  26.6s
[CV] END batch_size=80, epochs=50, learn_rate=0.2, momentum=0.4; total time=   3.4s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.6; total time=   1.9s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.8; total time=   4.9s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.9; total time=   2.4s
[CV] END batch_size=100, epochs=10, learn_rate=0.3, momentum=0.9; total time=   2.1s
[CV] END batch_size=10, epochs=10, learn_rate=0.001, momentum=0.4; total time=   5.1s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.0; total time=   2.9s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.8; total time=   4.9s
[CV] END batch_size=40, epochs=100, learn_rate=0.001, momentum=0.9; total time=   9.0s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.0; total time=   3.4s
[CV] END batch_size=60, epochs=50, learn_rate=0.2, momentum=0.

[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.2; total time=  26.6s
[CV] END batch_size=80, epochs=50, learn_rate=0.2, momentum=0.4; total time=   3.4s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.6; total time=   2.0s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.8; total time=   4.9s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.8; total time=   2.3s
[CV] END batch_size=100, epochs=10, learn_rate=0.3, momentum=0.9; total time=   2.1s
[CV] END batch_size=10, epochs=10, learn_rate=0.001, momentum=0.4; total time=   5.1s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.0; total time=   3.4s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.8; total time=   4.6s
[CV] END batch_size=40, epochs=100, learn_rate=0.001, momentum=0.9; total time=   9.1s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.0; total time=   3.1s
[CV] END batch_size=60, epochs=50, learn_rate=0.2, momentum=0.6;

[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.2; total time=  26.5s
[CV] END batch_size=20, epochs=10, learn_rate=0.3, momentum=0.6; total time=   3.1s
[CV] END batch_size=60, epochs=10, learn_rate=0.001, momentum=0.4; total time=   1.7s
[CV] END batch_size=20, epochs=10, learn_rate=0.01, momentum=0.9; total time=   2.9s
[CV] END batch_size=80, epochs=10, learn_rate=0.1, momentum=0.2; total time=   2.2s
[CV] END batch_size=40, epochs=10, learn_rate=0.001, momentum=0.9; total time=   2.4s
[CV] END batch_size=20, epochs=100, learn_rate=0.1, momentum=0.9; total time=  14.4s
[CV] END batch_size=60, epochs=50, learn_rate=0.01, momentum=0.9; total time=   3.3s
[CV] END batch_size=60, epochs=100, learn_rate=0.2, momentum=0.2; total time=   7.6s
[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.8; total time=   2.3s
[CV] END batch_size=40, epochs=100, learn_rate=0.3, momentum=0.8; total time=   8.0s
[CV] END batch_size=10, epochs=100, learn_rate=0.2, momentum=0.2; 

[CV] END batch_size=40, epochs=50, learn_rate=0.2, momentum=0.2; total time=   5.2s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.9; total time=  25.9s
[CV] END batch_size=100, epochs=50, learn_rate=0.1, momentum=0.4; total time=   3.1s
[CV] END batch_size=10, epochs=50, learn_rate=0.3, momentum=0.2; total time=  13.5s
[CV] END batch_size=60, epochs=100, learn_rate=0.3, momentum=0.6; total time=   6.6s
[CV] END batch_size=20, epochs=100, learn_rate=0.3, momentum=0.4; total time=  13.8s
[CV] END batch_size=100, epochs=50, learn_rate=0.01, momentum=0.4; total time=   3.7s
[CV] END batch_size=20, epochs=50, learn_rate=0.2, momentum=0.4; total time=   8.5s
[CV] END batch_size=80, epochs=10, learn_rate=0.001, momentum=0.4; total time=   2.6s
[CV] END batch_size=80, epochs=50, learn_rate=0.01, momentum=0.0; total time=   4.4s
[CV] END batch_size=100, epochs=100, learn_rate=0.2, momentum=0.9; total time=   8.5s
[CV] END batch_size=100, epochs=100, learn_rate=0.001, momentum=0

[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   5.1s
[CV] END batch_size=100, epochs=10, learn_rate=0.2, momentum=0.2; total time=   1.9s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.6; total time=   2.6s
[CV] END batch_size=80, epochs=50, learn_rate=0.3, momentum=0.8; total time=   4.3s
[CV] END batch_size=60, epochs=100, learn_rate=0.1, momentum=0.0; total time=   7.6s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   2.1s
[CV] END batch_size=10, epochs=10, learn_rate=0.01, momentum=0.0; total time=   4.3s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=  25.2s
[CV] END batch_size=80, epochs=10, learn_rate=0.01, momentum=0.9; total time=   1.6s
[CV] END batch_size=100, epochs=10, learn_rate=0.1, momentum=0.4; total time=   2.6s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.4; total time=   6.1s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.0;